In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
#     print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
from sklearn.utils import shuffle

fashion_mnist = keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [3]:
# suffle_train_data = [[x,y] for x,y in zip(X_train, y_train)]
# suffle_train_data = shuffle(suffle_train_data, random_state = 42)
# X_train = [n[0] for n in suffle_train_data]
# y_train = [n[1] for n in suffle_train_data]

# suffle_test_data = [[x,y] for x,y in zip(X_test, y_test)]
# suffle_test_data = shuffle(suffle_test_data, random_state = 42)
# X_test = [n[0] for n in suffle_test_data]
# y_test = [n[1] for n in suffle_test_data]

In [3]:
X_train = X_train /255
y_train = y_train.astype(float)

In [7]:
y_train[1]

0.0

In [4]:
def make_model():
    from functools import partial
    DefaultConv2D = partial(keras.layers.Conv2D,
                          kernel_size = 2, activation = 'selu', 
                           padding = 'SAME', kernel_initializer = 'lecun_normal')
    
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(64, (3,3), kernel_initializer = 'lecun_normal', activation = 'selu',
                                 input_shape = [28, 28, 1]))
    model.add(keras.layers.BatchNormalization())
    model.add(DefaultConv2D(filters = 32))
#     model.add(DefaultConv2D(filters = 32))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPooling2D((2,2)))
#     model.add(keras.layers.Dropout(rate=0.5))
    model.add(DefaultConv2D(filters = 64))
    model.add(keras.layers.BatchNormalization())
    model.add(DefaultConv2D(filters = 64))
#     model.add(DefaultConv2D(filters = 64))
    model.add(keras.layers.BatchNormalization())
#     model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(DefaultConv2D(filters = 128))
    model.add(keras.layers.BatchNormalization())
    model.add(DefaultConv2D(filters = 128))
#     model.add(DefaultConv2D(filters = 128))
    model.add(keras.layers.BatchNormalization())
#     model.add(keras.layers.Dropout(rate=0.5))

#     model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Flatten())
#     model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(64, kernel_initializer = 'lecun_normal', activation = 'selu'))
    model.add(keras.layers.BatchNormalization())
#     model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.Dense(32, kernel_initializer = 'lecun_normal', activation = 'selu'))
#     model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(10, activation = "softmax"))

    model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics = ['accuracy'])
    
    return model

In [5]:
def k_folding(num_epochs):
    early_stop_patiences = 5
    k = 10
    num_val = len(X_train) // k
    all_scores = []
    
    np.random.seed(42)
    tf.random.set_seed(42)

    #early stopping
    early_stopping_cb = keras.callbacks.EarlyStopping(patience = early_stop_patiences,
                                                     restore_best_weights = True,
                                                     monitor = 'val_accuracy',
                                                     mode = 'auto')

    #learning_rate down
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor = 0.5, patience = 10)
    
    for i in range(k):
        #callback
        checkpoint_cb = keras.callbacks.ModelCheckpoint(f"my_proj01_model_{i}.h5",
                                                   save_best_only = True)
        print('processing fold #', i)

        X_val = X_train[i * num_val: (i + 1) * num_val]
        y_val = y_train[i * num_val: (i + 1) * num_val]
        X_train_part = np.concatenate(
            [X_train[:i * num_val],
            X_train[(i + 1) * num_val:]],
            axis = 0)
        y_train_part = np.concatenate(
            [y_train[:i * num_val],
            y_train[(i + 1) * num_val:]],
            axis = 0)

        model = make_model()
        model.fit(X_train_part, y_train_part,
                 epochs = num_epochs, validation_data = (X_val, y_val),
                 batch_size = 8,
                 callbacks = [early_stopping_cb, checkpoint_cb])
        
        model = keras.models.load_model(f"my_proj01_model_{i}.h5")
        val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose = 0)
        all_scores.append(val_accuracy)
        print(f"{val_accuracy}\n")
    print(all_scores)
    print(f"score : {np.mean(all_scores)}")

In [6]:
k_folding(200)

processing fold # 0
Epoch 1/200


ValueError: in user code:

    C:\Users\kyeon\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:850 train_function  *
        return step_function(self, iterator)
    C:\Users\kyeon\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:840 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\kyeon\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\kyeon\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\kyeon\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\kyeon\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:833 run_step  **
 

In [ ]:
def make2_model():
    from functools import partial
    DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=2, strides=1,
                            padding="SAME", use_bias=False)

    class ResidualUnit(keras.layers.Layer):
        def __init__(self, filters, strides=1, activation="selu", **kwargs):
            super().__init__(**kwargs)
            self.activation = keras.activations.get(activation)
            self.main_layers = [
                DefaultConv2D(filters, strides=strides),
                keras.layers.BatchNormalization(),
                self.activation,
                DefaultConv2D(filters),
                keras.layers.BatchNormalization()]
            self.skip_layers = []
            if strides > 1:
                self.skip_layers = [
                    DefaultConv2D(filters, kernel_size=1, strides=strides),
                    keras.layers.BatchNormalization()]

        def call(self, inputs):
            Z = inputs
            for layer in self.main_layers:
                Z = layer(Z)
            skip_Z = inputs
            for layer in self.skip_layers:
                skip_Z = layer(skip_Z)
            return self.activation(Z + skip_Z)

    model = keras.models.Sequential()
    model.add(DefaultConv2D(64, kernel_size=2, strides=1,
                            input_shape=[28, 28, 1]))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("selu"))
    model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))
    prev_filters = 8
    for filters in [64] * 2 + [128] * 2 + [256] * 2 + [512] * 2:
        strides = 1 if filters == prev_filters else 2
        model.add(ResidualUnit(filters, strides=strides))
        prev_filters = filters
    model.add(keras.layers.GlobalAvgPool2D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(10, activation="softmax"))
    
    model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics = ['accuracy'])
    
    return model

In [ ]:
def k_folding(num_epochs):
    early_stop_patiences = 5
    k = 10
    num_val = len(X_train) // k
    all_scores = []
    
    np.random.seed(42)
    tf.random.set_seed(42)

    #early stopping
    early_stopping_cb = keras.callbacks.EarlyStopping(patience = early_stop_patiences,
                                                     restore_best_weights = True,
                                                     monitor = 'val_accuracy',
                                                     mode = 'auto')

    #learning_rate down
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor = 0.5, patience = 10)
    
    for i in range(k):
        #callback
        checkpoint_cb = keras.callbacks.ModelCheckpoint(f"my_proj01_model_{i}.h5",
                                                   save_best_only = True)
        print('processing fold #', i)

        X_val = X_train[i * num_val: (i + 1) * num_val]
        y_val = y_train[i * num_val: (i + 1) * num_val]
        X_train_part = np.concatenate(
            [X_train[:i * num_val],
            X_train[(i + 1) * num_val:]],
            axis = 0)
        y_train_part = np.concatenate(
            [y_train[:i * num_val],
            y_train[(i + 1) * num_val:]],
            axis = 0)

        model = make2_model()
        model.fit(X_train_part, y_train_part,
                 epochs = num_epochs, validation_data = (X_val, y_val),
                 batch_size = 64,
                 callbacks = [early_stopping_cb, checkpoint_cb])
        
        model = keras.models.load_model(f"my_proj01_model_{i}.h5")
        val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose = 0)
        all_scores.append(val_accuracy)
        print(f"{val_accuracy}\n")
    print(all_scores)
    print(f"score : {np.mean(all_scores)}")

In [ ]:
k_folding(40)

In [9]:
# def model_run(num_epochs, num_batch_size):
#     early_stop_patiences = 15
#     keras.backend.clear_session()
#     np.random.seed(42)
#     tf.random.set_seed(42)
    
#     model = make_model()
#     model.summary()
    
#     #callback
#     checkpoint_cb = keras.callbacks.ModelCheckpoint("my_mnist_model.h5",
#                                                    save_best_only = True)
#     #early stopping
#     early_stopping_cb = keras.callbacks.EarlyStopping(patience = early_stop_patiences,
#                                                      restore_best_weights = True)
    
#     #learning_rate down
#     lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor = 0.5, patience = 5)
    
#     model.fit(X_train, y_train, epochs = num_epochs, batch_size = 2 ** num_batch_size,
#               validation_data = (X_valid, y_valid),
#              callbacks = [lr_scheduler, early_stopping_cb, checkpoint_cb])
    
#     model = keras.models.load_model("my_mnist_model.h5")
#     test_loss, test_acc = model.evaluate(X_test, y_test)
#     print(test_acc)
#     print(early_stopping_cb.stopped_epoch - early_stop_patiences + 1)

In [10]:
# model_run(500, 7)
# #model_run(epoch 수, 2의 n제곱)

In [11]:
#0.9054999947547913
#0.8917999863624573
#500 epoch, 1024 batch 0.9107999801635742

#selu
#500 epoch, 1024 batch 0.8946999907493591
#500 epoch, 1024 batch 0.8986999988555908

#500, 256 0.9146000146865845

#0.9126999974250793
#0.9232000112533569
#0.929099977016449(32 32 32 64 64 64 128 128 128)
#0.9325000047683716(64 32 64 64 128 128)
#0.9358999729156494(64 32 64 64 128 128) batch 2**7


In [12]:
# def make2_model():
#     from functools import partial
#     DefaultConv2D = partial(keras.layers.Conv2D,
#                           kernel_size = 2, activation = 'selu', 
#                            padding = 'SAME', kernel_initializer = 'lecun_normal')
    
#     model = keras.models.Sequential()
#     model.add(keras.layers.Conv2D(64, (3,3), kernel_initializer = 'lecun_normal', activation = 'selu',
#                                  input_shape = [28, 28, 1]))
#     model.add(keras.layers.Dropout(rate = 0.5))
#     model.add(DefaultConv2D(filters = 64))
# #     model.add(DefaultConv2D(filters = 32))
#     model.add(keras.layers.MaxPooling2D((2,2)))
#     model.add(keras.layers.BatchNormalization())
# #     model.add(keras.layers.Dropout(rate=0.5))
#     model.add(DefaultConv2D(filters = 64))
#     model.add(DefaultConv2D(filters = 64))
# #     model.add(DefaultConv2D(filters = 64))
#     model.add(keras.layers.BatchNormalization())
# #     model.add(keras.layers.Dropout(rate=0.5))
#     model.add(keras.layers.MaxPooling2D((2,2)))
#     model.add(DefaultConv2D(filters = 128))
#     model.add(DefaultConv2D(filters = 128))
# #     model.add(DefaultConv2D(filters = 128))
#     model.add(keras.layers.BatchNormalization())
# #     model.add(keras.layers.Dropout(rate=0.5))

# #     model.add(keras.layers.GlobalAveragePooling2D())
#     model.add(keras.layers.Flatten())
#     model.add(keras.layers.Dropout(rate=0.5))
#     model.add(keras.layers.Dense(64, kernel_initializer = 'lecun_normal', activation = 'selu'))
#     model.add(keras.layers.BatchNormalization())
# #     model.add(keras.layers.Dropout(rate=0.5))
#     model.add(keras.layers.Dense(32, kernel_initializer = 'lecun_normal', activation = 'selu'))
# #     model.add(keras.layers.Dropout(rate=0.5))
#     model.add(keras.layers.BatchNormalization())
#     model.add(keras.layers.Dense(10, activation = "softmax"))

#     model.compile(optimizer='adam',
#              loss='categorical_crossentropy',
#              metrics = ['accuracy'])
    
#     return model

In [13]:
# def model2_run(num_epochs, num_batch_size):
#     early_stop_patiences = 15
#     keras.backend.clear_session()
#     np.random.seed(42)
#     tf.random.set_seed(42)
    
#     model = make2_model()
#     model.summary()
    
#     #callback
#     checkpoint_cb = keras.callbacks.ModelCheckpoint("my_mnist_model_2.h5",
#                                                    save_best_only = True)
#     #early stopping
#     early_stopping_cb = keras.callbacks.EarlyStopping(patience = early_stop_patiences,
#                                                      restore_best_weights = True)
    
#     #learning_rate down
#     lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor = 0.5, patience = 5)
    
#     model.fit(X_train, y_train, epochs = num_epochs, batch_size = 2 ** num_batch_size,
#               validation_data = (X_valid, y_valid),
#              callbacks = [lr_scheduler, early_stopping_cb, checkpoint_cb])
    
#     model = keras.models.load_model("my_mnist_model_2.h5")
#     test_loss, test_acc = model.evaluate(X_test, y_test)
#     print(test_acc)
#     print(early_stopping_cb.stopped_epoch - early_stop_patiences + 1)

In [14]:
# model2_run(500, 7)

In [15]:
# def make3_model():
#     from functools import partial
#     DefaultConv2D = partial(keras.layers.Conv2D,
#                           kernel_size = 3, activation = 'selu', 
#                            padding = 'SAME', kernel_initializer = 'lecun_normal')
    
#     model = keras.models.Sequential()
#     model.add(keras.layers.Conv2D(128, (2,2), kernel_initializer = 'lecun_normal', activation = 'selu',
#                                  input_shape = (28, 28, 1)))
#     model.add(DefaultConv2D(filters = 128))
#     model.add(keras.layers.MaxPooling2D((2,2)))
#     model.add(keras.layers.BatchNormalization())
# #     model.add(keras.layers.Dropout(rate=0.5))
#     model.add(DefaultConv2D(filters = 256))
#     model.add(DefaultConv2D(filters = 256))
# #     model.add(keras.layers.Dropout(rate=0.5))
#     model.add(keras.layers.BatchNormalization())
#     model.add(keras.layers.MaxPooling2D((2,2)))
#     model.add(DefaultConv2D(filters = 512))
#     model.add(DefaultConv2D(filters = 512))
# #     model.add(keras.layers.Dropout(rate=0.5))
#     model.add(keras.layers.BatchNormalization())

# #     model.add(keras.layers.GlobalAveragePooling2D())
#     model.add(keras.layers.Flatten())
#     model.add(keras.layers.Dense(256, kernel_initializer = 'lecun_normal', activation = 'selu'))
#     model.add(keras.layers.BatchNormalization())
#     model.add(keras.layers.Dropout(rate=0.5))
#     model.add(keras.layers.Dense(128, kernel_initializer = 'lecun_normal', activation = 'selu'))
#     model.add(keras.layers.Dropout(rate=0.5))
#     model.add(keras.layers.BatchNormalization())
#     model.add(keras.layers.Dense(10, activation = "softmax"))

#     model.compile(optimizer='adam',
#              loss='categorical_crossentropy',
#              metrics = ['accuracy'])
    
#     return model

In [16]:
# def model3_run(num_epochs, num_batch_size):
#     keras.backend.clear_session()
#     np.random.seed(42)
#     tf.random.set_seed(42)
    
#     model = make3_model()
    
#     #callback
#     checkpoint_cb = keras.callbacks.ModelCheckpoint("my_mnist_model_3.h5",
#                                                    save_best_only = True)
#     #early stopping
#     early_stopping_cb = keras.callbacks.EarlyStopping(patience = 20,
#                                                      restore_best_weights = True)
    
#     #learning_rate down
#     lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor = 0.5, patience = 5)
    
#     model.fit(X_train, y_train, epochs = num_epochs, batch_size = num_batch_size,
#               validation_data = (X_valid, y_valid),
#              callbacks = [lr_scheduler, early_stopping_cb, checkpoint_cb])
    
#     model = keras.models.load_model("my_mnist_model_3.h5")
#     test_loss, test_acc = model.evaluate(X_test, y_test)
#     print(test_acc)

In [17]:
# model3_run(500, 256)